# Worksheet 1: Types

1. What happens if you try to instantiate a cme257int with a float?  Why did this happen?

2. Create a type called "cme257OrderedPair" with the following properties:
    1. It is a child of cme257abstract
    2. It has two fields: "a" and "b"
    3. The fields "a" and "b" are the same parameterized type
    
3. How do you create a Complex number in Julia?  Investigate the resulting type:
    1. Is it a primitive type?  If not, what fields does it have?
    2. What else do you notice?  Is it immutable?  Is it parameterized?

In [9]:
abstract type cme257abstract end

In [10]:
# an abstract type for our real number types
abstract type cme257real <: cme257abstract end

struct cme257int <: cme257real # <: denotes "child of"
    x::Int64 # :: tells us exactly what type x should be
end
struct cme257float <: cme257real
    x::Float64
end

In [11]:
#a0 = cme257int(5.0)
a1 = cme257int(5.5)

InexactError: InexactError: Int64(5.5)

In [12]:
struct cme257OrderedPair{T} <: cme257abstract
    a::T
    b::T
end

In [13]:
op1 = cme257OrderedPair("a","b")
op1.a

"a"

In [14]:
z = 5.0 + 1.0im

5.0 + 1.0im

In [15]:
typeof(z)

Complex{Float64}

In [16]:
# fields of complex numbers
@show z.re
@show z.im
z.re = 3.0 # immutable
;

z.re = 5.0
z.im = 1.0


ErrorException: setfield! immutable struct of type Complex cannot be changed

# Worksheet 2: Functions

1. Overload the addition and multiplication operators to do element-wise addition and multiplication on cme257OrderedPair.

2. Overload addition and multiplication to work with cme257ff{N,T} for N any positive integer (don't worry about checking positivity).

(a +/* b).val = a.val +/* b.val mod N

Generally, this gives this type the structure of a ring.  If N is prime, the type has the structure of a field.

3. Modify ```yell_my_type``` to yell all types correctly.

In [17]:
import Base.+, Base.*

x = cme257OrderedPair(2, 3)
y = cme257OrderedPair(3, 4)

function +(x::cme257OrderedPair{T}, y::cme257OrderedPair{T}) where T
    return cme257OrderedPair(x.a + y.a, x.b + y.b) 
end

@show x + y

function *(x::cme257OrderedPair{T}, y::cme257OrderedPair{T}) where T
    return cme257OrderedPair(x.a * y.a, x.b * y.b)
end

@show x * y
;

x + y = cme257OrderedPair{Int64}(5, 7)
x * y = cme257OrderedPair{Int64}(6, 12)


In [18]:
# we'll use this for finite fields with characteristic N
struct cme257ff{N, T<:Integer} #<: cme257abstract
    val::T
    # override the default constructor to store things mod N
    function cme257ff{N,T}(val::T) where {N,T<:Integer} 
        return new(mod(val,N))
    end
end

# You'll also see the N convention in Julia with types like Array{T,N}, where T is a type and N is a number

# This will create constructors where the type of the value is inferred
function cme257ff{N}(val::T) where {N, T}
    return cme257ff{N,T}(val)
end
# this will create constructors where signed integers are converted to unsigned integers
function cme257ff{N,T}(val::T) where {N, T<:Signed} 
    return cme257ff{N}(Unsigned(val))
end
# Note that the above are examples of creating functions

function Base.show(io::IO, x::cme257ff{N,T}) where {N,T}
    print(io, x.val, " mod ", N, " (", T, ")")
    return
end

In [19]:
function *(a::cme257ff{N,T}, b::cme257ff{N,T}) where {N,T}
    cme257ff{N,T}(T(mod(a.val * b.val, N)))
end

function +(a::cme257ff{N,T}, b::cme257ff{N,T}) where {N,T}
    cme257ff{N,T}(T(mod(a.val + b.val,N)))
end

x = cme257ff{5}(2)
y = cme257ff{5}(4)
@show x*y # 2 * 4 = 8 = 3 mod 5
@show x+y # 2 + 4 = 6 = 1 mod 5
;

x * y = 3 mod 5 (UInt64)
x + y = 1 mod 5 (UInt64)


In [20]:
# note that with our cme257ff type, we can't add or multiply elements of different rings
x = cme257ff{5}(2)
y = cme257ff{7}(4)
@show x+y
;

MethodError: MethodError: no method matching +(::cme257ff{5,UInt64}, ::cme257ff{7,UInt64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(::cme257ff{N,T}, !Matched::cme257ff{N,T}) where {N, T} at In[19]:6

In [21]:
# we also need the same underlying type
x = cme257ff{5}(UInt16(2))
y = cme257ff{5}(UInt32(4))
@show x*y
;

MethodError: MethodError: no method matching *(::cme257ff{5,UInt16}, ::cme257ff{5,UInt32})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(::cme257ff{N,T}, !Matched::cme257ff{N,T}) where {N, T} at In[19]:2

In [22]:
function yell_my_type(x::T) where T
   println(uppercase("MY TYPE IS $(T)"))
end

yell_my_type(5//7)

MY TYPE IS RATIONAL{INT64}
